<div style="text-align: center;">
    <h1>Base SIRENE</h1>
</div>


La **base Sirene** est le fournisseur des données d’identité des entreprises et des établissements. 
Elle fait partie des données de référence du Service public de la donnée mis en place par la loi pour une République numérique.
Voir plus d'informations sur le site de **[l'Insee](https://www.insee.fr/fr/information/3591226#:~:text=Elle%20donne%20acc%C3%A8s%20aux%20donn%C3%A9es,r%C3%A9pertoire%20interadministratif%20Sirene%20depuis%201973.)**.

Le site [sirene.fr](https://www.sirene.fr/sirene/public/accueil) permet de créer en ligne des fichiers comprenant au maximum 200 000 établissements, ce qui n'est pas assez pour nos besoins ici, il faut donc tout télécharger sur [data gouv](https://www.data.gouv.fr/fr/datasets/base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret/).

Le but de ce notebook est de télécharger cette base de données et de la nettoyer pour obtenir les informations suivantes :

- **SIRET** : code d'identification d'un établissement. On distingue ici les entreprises (identifiées par leur numéro SIREN) des établissements (SIRET = SIREN + NIC)
- **code NAF/APE** : activité principale de l'établissement renseignée à l'INPI
- **code commune INSEE** : code de la commune pour pouvoir localiser les entreprises
- **dénomination  / nom commercial** : nom de l'entreprise 



### Import des librairies

In [6]:
! pip install -q matplotlib seaborn duckdb


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import os 
import pandas as pd
import hashlib
from tqdm import tqdm
import subprocess

# 1. Téléchargement des données

On télécharge les données via ce [lien permanent](https://www.data.gouv.fr/fr/datasets/r/0651fb76-bcf3-4f6a-a38d-bc04fa708576) aux serveurs de DataGouv.

La documentation relative au jeu de données est disponible [ici](https://static.data.gouv.fr/resources/base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret/20230523-095315/description-fichier-stocketablissement.pdf).

In [6]:

os.system("wget -q -P '../Données nationales/' https://www.data.gouv.fr/fr/datasets/r/0651fb76-bcf3-4f6a-a38d-bc04fa708576")
os.system("wget -q -P '../Données nationales/' https://static.data.gouv.fr/resources/base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret/20230523-095315/description-fichier-stocketablissement.pdf")

0

Verification que le téléchargement s'est correctement effectué.

In [7]:
# id du document
id_doc = '0651fb76-bcf3-4f6a-a38d-bc04fa708576'

# hash du document
hash_datagouv = "c709c26eefa07043f5fb5cf74816ec01dc4739f37589a1c02a70ab6583cb37ae"

# on hash le document (sha256)
hash_local = hashlib.sha256(open('../Données nationales/'+id_doc, 'rb').read()).hexdigest()

# on vérifie que le hash du document téléchargé correspond au hash connu
if hash_datagouv == hash_local:
    print("Le fichier a été téléchargé correctement")
else:
    print("Le fichier téléchargé est corrompu")

Le fichier a été téléchargé correctement


Décompression du fichier 

In [8]:
os.system("unzip -q -o '../Données nationales/"+id_doc+"' -d '../Données nationales/'")
os.remove('../Données nationales/'+id_doc)

Conversion en parquet pour optimiser la mémoire et le traitement

In [122]:
df = pd.read_csv('../Données nationales/StockEtablissement_utf8.csv',nrows=100)

# la documentation indique qu'il y a 48 colonnes
if df.shape[1] == 48:
    print("Il y a bien 48 colonnes dans le fichier")
else:
    print("Vérifier la lecture du fichier")

# poids total du fichier en Go
poids = os.path.getsize('../Données nationales/StockEtablissement_utf8.csv')/1024**3
print("Le fichier pèse {:.2f} Go".format(poids))

Il y a bien 48 colonnes dans le fichier
Le fichier pèse 6.37 Go


Définition des variables que l'on souhaite garder et des transformations à appliquer

In [127]:
# nom des variables d'intérêt
desired_col_list =["siret","numeroVoieEtablissement","typeVoieEtablissement","libelleVoieEtablissement",
                   "codePostalEtablissement","libelleCommuneEtablissement","codeCommuneEtablissement",
                    "activitePrincipaleEtablissement","denominationUsuelleEtablissement","etatAdministratifEtablissement",
                    "enseigne1Etablissement","enseigne2Etablissement","enseigne3Etablissement",
                    "statutDiffusionEtablissement","nomenclatureActivitePrincipaleEtablissement"]


# position des colonnes à enlever
col_to_drop = [index for index, nom in enumerate(df.columns) if nom not in desired_col_list]

# position des colonnes à garder
col_to_keep = [index for index, nom in enumerate(df.columns) if nom in desired_col_list]

# transformations à appliquer sur les colonnes
# en key on met le nom de la colonne dans le fichier final, en value on met le nom de la / des colonnes dans le fichier initial
# si la value est une liste, on concatène les colonnes avec un espace
col_transform = {"siret":"siret",
                 "adresseEtablissement": ["numeroVoieEtablissement","typeVoieEtablissement","libelleVoieEtablissement",
                                            "codePostalEtablissement","libelleCommuneEtablissement"],
                "codeCommuneEtablissement":"codeCommuneEtablissement",
                "activitePrincipaleEtablissement":"activitePrincipaleEtablissement",
                "enseigneEtablissement":["enseigne1Etablissement","enseigne2Etablissement","enseigne3Etablissement"],  
                "denominationUsuelleEtablissement":"denominationUsuelleEtablissement",
                "etatAdministratifEtablissement":"etatAdministratifEtablissement",
                "statutDiffusionEtablissement":"statutDiffusionEtablissement",
                "nomenclatureActivitePrincipaleEtablissement":"nomenclatureActivitePrincipaleEtablissement"}



# fonction qui permet de transformer les colonnes
def transform_col(df,col_to_drop,col_transform):

    # on enlève les colonnes qui ne nous intéressent pas
    df = df.drop(df.columns[col_to_drop],axis=1)

    # on transforme les colonnes
    for key, value in col_transform.items():

        if isinstance(value, list):

            # join columns with a space if value is a list, replace NaN with empty string
            df[key] = df[value].apply(lambda x: ' '.join(x.dropna().astype(str)),axis=1)

        else:
            df[key] = df[value]

    # on ne garde que les colonne finales
    df = df[list(col_transform.keys())]

    # on elimine les doublons
    df = df.drop_duplicates()

    # on elimine les etablissements fermés
    df = df[df["etatAdministratifEtablissement"] == "A"]

    # on enlève la colonne etatAdministratifEtablissement
    df = df.drop(["etatAdministratifEtablissement"],axis=1)

    # definition des types de variables
    df = df.astype({"siret":int,
                    "adresseEtablissement":str,
                    "codeCommuneEtablissement":str,
                    "activitePrincipaleEtablissement":str,
                    "enseigneEtablissement":str,
                    "denominationUsuelleEtablissement":str,
                    "statutDiffusionEtablissement":str,
                    "nomenclatureActivitePrincipaleEtablissement":str})

    return df


Test sur un échantillon

In [124]:
%%time

# test de la fonction sur un échantillon
df = pd.read_csv('../Données nationales/StockEtablissement_utf8.csv',nrows=100000,low_memory=False)

df = transform_col(df,col_to_drop,col_transform)

# colonnes finales = colonnes du dictionnaire - colonne etatAdministratifEtablissement
final_col = list(col_transform.keys())
final_col.remove("etatAdministratifEtablissement")
                                           

# on vérifie que les colonnes finales sont bien dans le dataframe
if set(final_col).issubset(set(df.columns)):
    print("Les colonnes finales sont bien dans le dataframe")
else:
    print("Les colonnes finales ne sont pas dans le dataframe")



Les colonnes finales sont bien dans le dataframe
CPU times: user 7.36 s, sys: 95.7 ms, total: 7.46 s
Wall time: 7.47 s


Choix du nombre de chunks : d'après le test précédent, il est cohérent de choisir des chunks de 100 000 lignes

In [125]:
# Command to get the number of lines in the CSV file
command = "wc -l '../Données nationales/StockEtablissement_utf8.csv' | awk '{print $1}'"

# Run the command and capture the output
output = subprocess.check_output(command, shell=True)

# Convert the output to an integer
number_of_lines = int(output.decode('utf-8').strip())

# Calculate the number of chunks (assuming chunk size of 100000)
chunk_size = 100000
number_of_chunks = number_of_lines // chunk_size

# Print or use the variable as needed
print(f"Number of lines: {number_of_lines}")
print(f"Number of chunks: {number_of_chunks}")

Number of lines: 37768658
Number of chunks: 377


Transformation du csv et sauvegarde en parquet

In [130]:
# on lit le csv en chunks
csv_reader = pd.read_csv('../Données nationales/StockEtablissement_utf8.csv',chunksize=chunk_size,low_memory=False)

# on initialise le dataframe final
df_final = pd.DataFrame(columns=final_col)

# list des chunks
chunks_list = [transform_col(chunk, col_to_drop, col_transform) for chunk in tqdm(csv_reader)]

# on enregistre le dataframe final
df_final = pd.concat(chunks_list,ignore_index=True)

# on enregistre le dataframe final
df_final.to_parquet('../Données nationales/StockEtablissement_utf8.parquet',compression='GZIP')

378it [1:18:36, 12.48s/it]


Vérification du bien fondé de l'opération précédente et suppression du csv

In [139]:
# poids des fichiers en Go
poids = os.path.getsize('../Données nationales/StockEtablissement_utf8.parquet')/1024**3
print("Le fichier parquet pèse {:.2f} Go".format(poids))
poids = os.path.getsize('../Données nationales/StockEtablissement_utf8.csv')/1024**3
print("Le fichier csv pèse {:.2f} Go".format(poids))


Le fichier parquet pèse 0.34 Go
Le fichier csv pèse 6.37 Go
